Modules nécessaires : 

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

from HiggsML.ingestion import Ingestion
from HiggsML.datasets import download_dataset
from sample_code_submission.neural_network import NeuralNetwork
from sys import path
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from itertools import product
from numpy.random import RandomState
import warnings
import os
import sys

warnings.filterwarnings("ignore")

Charger les données : 

In [2]:
data = download_dataset(
    "blackSwan_data"
)  # change to "blackSwan_data" for the actual data

# load train set
data.load_train_set()
data_set = data.get_train_set()

2025-06-04 14:09:38,578 - HiggsML.datasets     - INFO     - Handling as dataset name: blackSwan_data
2025-06-04 14:09:38,580 - HiggsML.datasets     - INFO     - Current working directory: c:\Users\leand\Desktop\EI Higgs\Higgs_collaboration_A
2025-06-04 14:09:38,633 - HiggsML.datasets     - INFO     - Total rows: 2000000
2025-06-04 14:09:38,635 - HiggsML.datasets     - INFO     - Test size: 600000
2025-06-04 14:09:38,723 - HiggsML.datasets     - INFO     - Selected train size: 1400000
2025-06-04 14:09:39,959 - HiggsML.datasets     - INFO     - Data loaded successfully


In [3]:
target = data_set["labels"]
weights = data_set["weights"]
detailed_label = data_set["detailed_labels"]
keys = np.unique(detailed_label)

Direction

In [4]:
root_dir = os.getcwd()
print("Root directory is", root_dir)
submission_dir = os.path.join(root_dir, "sample_code_submission")

# The directory where results and other outputs from the participant's code will be written
output_dir = os.path.join(root_dir, "sample_result_submission")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Root directory is c:\Users\leand\Desktop\EI Higgs\Higgs_collaboration_A


Import Submission Model

In [5]:
path.append(submission_dir)
from model import Model

Testing Settings

In [6]:
TEST_SETTINGS = {
    "systematics": {  # Systematics to use
        "tes": False,  # tau energy scale
        "jes": False,  # jet energy scale
        "soft_met": False,  # soft term in MET
        "ttbar_scale": False,  # W boson scale factor
        "diboson_scale": False,  # Diboson scale factor
        "bkg_scale": False,  # Background scale factor
    },
    "num_pseudo_experiments": 25,  # Number of pseudo-experiments to run per set
    "num_of_sets": 25,  # Number of sets of pseudo-experiments to run
}

RANDOM_SEED = 42


In [7]:
test_settings = TEST_SETTINGS.copy()

random_state = np.random.RandomState(RANDOM_SEED)
test_settings["ground_truth_mus"] = (
    random_state.uniform(0.1, 3, test_settings["num_of_sets"])
).tolist()

random_settings_file = os.path.join(output_dir, "test_settings.json")
with open(random_settings_file, "w") as f:
    json.dump(test_settings, f)

Boucle sur "Ingestion" en faisant varier epochs

Fonction bouclée: 

Significance

In [ ]:

def optimization(train_data): 
    activation_fn = {"relu", "sigmoid"}
    sigmax=0
    L=[]
    for fn in activation_fn :
        ingestion = Ingestion(data)
        # initialize submission
        ingestion.init_submission(Model, "NN")
        ingestion.model.model.activation = fn
        print(ingestion.model.model.activation)
        ingestion.fit_submission()
        sig1 = ingestion.model.model.significance_2(test_labels=ingestion.model.training_set["labels"],test_weights=ingestion.model.training_set["weights"])
        L.append((fn, sig1))
        #visualisation des données :
        if sig1>sigmax: 
            sigmax = sig1
            fn_max = fn
    return sigmax, fn_max, L



Il y a plusieurs problématique : 
Comment modifier une variable epochs de notre 

Lancement de la boucle

In [9]:
optimization(data_set)

2025-06-04 14:09:41,112 - HiggsML.ingestion    - INFO     - Initializing Submmited Model
2025-06-04 14:09:41,143 - HiggsML.datasets     - INFO     - Selected train size: 80000
2025-06-04 14:09:41,635 - HiggsML.datasets     - INFO     - Data loaded successfully
2025-06-04 14:09:41,667 - HiggsML.datasets     - INFO     - Selected train size: 10000


Training Data:  (80000, 28)
Training Labels:  (80000,)
Training Weights:  (80000,)
sum_signal_weights:  678.1138992144495
sum_bkg_weights:  105040.88610078552


2025-06-04 14:09:42,142 - HiggsML.datasets     - INFO     - Data loaded successfully
2025-06-04 14:09:42,164 - HiggsML.datasets     - INFO     - Selected train size: 10000



Valid Data:  (10000, 28)
Valid Labels:  (10000,)
Valid Weights:  (10000,)
sum_signal_weights:  680.719996621049
sum_bkg_weights:  105038.28000337898


2025-06-04 14:09:42,562 - HiggsML.datasets     - INFO     - Data loaded successfully



Holdout Data:  (10000, 28)
Holdout Labels:  (10000,)
Holdout Weights:  (10000,)
sum_signal_weights:  677.7287697327586
sum_bkg_weights:  105041.27123026722
 
 
Training Data:  (80000, 28)
DEBUG: model_type = 'NN'


AttributeError: 'NeuralNetwork' object has no attribute 'activation'